In [13]:
import torch
import pandas as pd
import os, sys
import numpy as np

In [1]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

/home/marco/miniconda3/envs/hackaton2023/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%load_ext autoreload
%autoreload 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
data_tain_path = './dataset/train_set.csv'
data_test_path = './dataset/test_set.csv'
batch_size = 4

In [5]:
data_train = pd.read_csv(data_tain_path)
data_test = pd.read_csv(data_test_path)

In [6]:
data_train

,Job_offer,Label
0,"Openjobmetis SpA ricerca, per importante azien...",Java Developer
1,"La persona prescelta, diplomata o laureata in ...",Software Engineer
2,Sei un informatico o matematico con la passion...,Software Engineer
3,"Ti occuperai della progettazione, realizzazion...",Programmer
4,Stiamo cercando uno\una sviluppatore\sviluppat...,Programmer
...,...,...
1747,"* Test JUnit\n * Git, La Ibs Srl è alla ri...",Java Developer
1748,"La ricorsa, inserita all'interno di un team, l...",Programmer
1749,Sviluppatore Java Junior da inserire in attivi...,Java Developer
1750,La risorsa sarà inserita nei team di risorse d...,Web Developer


In [7]:
data_train_formatted = []
for pd_data_row in data_train.iloc:
    row = {'sentence': pd_data_row['Job_offer'], 'label': pd_data_row['Label']}
    data_train_formatted.append(row)

data_test_formatted = []
for pd_data_row in data_test.iloc:
    row = {'sentence': pd_data_row['Job_offer'], 'label': pd_data_row['Label']}
    data_test_formatted.append(row)

In [8]:
data_train_labels = sorted(list(set([r["label"] for r in data_train_formatted])))
data_test_labels = sorted(list(set([r["label"] for r in data_test_formatted])))

In [9]:
data_train_labels

['Java Developer',
 'Programmer',
 'Software Engineer',
 'System Analyst',
 'Web Developer']

In [10]:
data_test_labels

['Java Developer',
 'Programmer',
 'Software Engineer',
 'System Analyst',
 'Web Developer']

In [11]:
id_to_label = data_train_labels
label_to_id = {l:i for i,l in enumerate(id_to_label)}

In [15]:
correct = 0
for e in data_test_formatted:
    res = classifier(e['sentence'], data_test_labels)
    label = res['labels'][np.argmax(res['scores'])]
    if e['label'] == label:
        correct += 1
print(correct, len(data_test_formatted))